In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

df = pd.read_csv('dst-3.0_16_1_hh_database.csv',sep=';')
df['Ищет работу на должность:'].mode().iloc[0]

'Системный администратор'

In [32]:
# Перезагрузим исходные данные
df = pd.read_csv('dst-3.0_16_1_hh_database.csv', sep=';')

# Обработка ЗП
def extract_salary_and_currency(salary_str):
    if pd.isna(salary_str):
        return pd.Series([np.nan, np.nan])
    salary_str = salary_str.replace('\xa0', ' ')
    parts = salary_str.split()
    try:
        amount = int(parts[0].replace(' ', ''))
    except:
        return pd.Series([np.nan, np.nan])
    currency_raw = parts[1]
    currency_map = {
        'руб.': 'RUB',
        'грн.': 'UAH',
        'USD': 'USD',
        'EUR': 'EUR',
        'белруб': 'BYN',
        'KGS': 'KGS',
        'сум': 'UZS',
        'AZN': 'AZN',
        'KZT': 'KZT'
    }
    currency = currency_map.get(currency_raw, 'RUB')  # по умолчанию рубли
    return pd.Series([amount, currency])

df[['salary_amount', 'currency_iso']] = df['ЗП'].apply(extract_salary_and_currency)

# Обработка дат
df['Обновление резюме'] = pd.to_datetime(df['Обновление резюме'], format='%d.%m.%Y %H:%M')
df['date_resume'] = df['Обновление резюме'].dt.date

# Загрузка курсов
rates_df = pd.read_csv('ExchangeRates.csv')
rates_df['date'] = pd.to_datetime(rates_df['date'], dayfirst=True).dt.date

# Добавление RUB
rub_rows = pd.DataFrame({
    'date': rates_df['date'].unique(),
    'currency': 'RUB',
    'proportion': 1,
    'close': 1.0
})
rates_full = pd.concat([rates_df, rub_rows], ignore_index=True)

# Объединение
df = df.merge(
    rates_full[['currency', 'date', 'proportion', 'close']],
    left_on=['currency_iso', 'date_resume'],
    right_on=['currency', 'date'],
    how='left'
)

# Расчёт ЗП в рублях
df['ЗП (руб)'] = df['salary_amount'] * df['close'] / df['proportion']

# Медиана в тысячах рублей
round(df['ЗП (руб)'].median() / 1000)

/var/folders/wp/hfz02kwd711fdrnqxh4lx2tr0000gn/T/ipykernel_46030/2813611470.py:37: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  rates_df['date'] = pd.to_datetime(rates_df['date'], dayfirst=True).dt.date


58

In [26]:
million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград']

def extract_city(location_str):
    if pd.isna(location_str):
        return np.nan
    city_part = location_str.split(' , ')[0]
    if city_part == 'Москва':
        return 'Москва'
    elif city_part == 'Санкт-Петербург':
        return 'Санкт-Петербург'
    elif city_part in million_cities:
        return 'город-миллионник'
    else:
        return 'другие'

def extract_relocation(location_str):
    if pd.isna(location_str):
        return False
    parts = location_str.split(' , ')
    for part in parts[1:]:
        if 'переезд' in part or 'переехать' in part:
            return 'готов' in part or 'хочу' in part
    return False

def extract_business_trips(location_str):
    if pd.isna(location_str):
        return False
    parts = location_str.split(' , ')
    for part in parts[1:]:
        if 'командиров' in part:
            return 'готов' in part
    return False

df['Город'] = df['Город, переезд, командировки'].apply(extract_city)
df['Готовность к переезду'] = df['Город, переезд, командировки'].apply(extract_relocation)
df['Готовность к командировкам'] = df['Город, переезд, командировки'].apply(extract_business_trips)

df.drop(columns=['Город, переезд, командировки'], inplace=True)

round((df['Город'] == 'Санкт-Петербург').mean() * 100)


KeyError: 'Город, переезд, командировки'

In [24]:
round(df['ЗП (руб)'].median() / 1000)

60